In [1]:
import pyterrier as pt
import pyterrier_doc2query
from pyterrier.measures import *
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
#build the index
#files="C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/Datasets/msmarco/input/fulldocs-new.trec"
#indexer=pt.TRECCollectionIndexer("C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/Datasets/index",verbose=True,blocks=False)
#indexref=indexer.index(files)


In [3]:
#load the index
index_path = "C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/Datasets/index"
indexref=pt.IndexRef.of(index_path)
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 3213835
Number of terms: 16168096
Number of postings: 905088837
Number of fields: 0
Number of tokens: 2204592607
Field names: []
Positions:   false



In [4]:
#build new index
def iter_file(filename):
  import json
  with open(filename, 'rt') as file:
    for l in file:
      # assumes that each line contains 'docno', 'text' attributes
      # yields a dictionary for each json line
      yield json.loads(l)
files_d2q="C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/Datasets/msmarco/doc2query/input/docs.json"
indexer_d2q=pt.IterDictIndexer("C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/Datasets/index_d2q",verbose=True,blocks=False)
indexref_d2q=indexer_d2q.index(iter_file(files_d2q))
#uncommend below if already built
#index_path_d2q = "C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/Datasets/index"
#indexref_d2q=pt.IndexRef.of(index_path_d2q)
index_d2q = pt.IndexFactory.of(indexref_d2q)
print(index_d2q.getCollectionStatistics().toString())

Number of documents: 3213835
Number of terms: 29718814
Number of postings: 1074683429
Number of fields: 1
Number of tokens: 2916542609
Field names: [text]
Positions:   false



In [21]:
#bm25 baseline
topics_path = "C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/anserini/tools/topics-and-qrels/topics.msmarco-doc.test.txt"
qrels_path = "C:/Users/Th3o0/Desktop/University/MSC/IN4325 - Information Retrieval - Q3 2022/Project/anserini/tools/topics-and-qrels/qrels.dl19-doc.txt"

qrels = pt.io.read_qrels(qrels_path)
topics = pt.io.read_topics(topics_path,format="singleline")

bm25_base = pt.BatchRetrieve(index, wmodel="BM25")
#res_base = bm25_base.transform(topics)
#pt.Utils.evaluate(res_base, qrels, metrics = ['ndcg',nDCG@10,'map','mrt'])


23:51:43.242 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [22]:
#bm25 with query expansion
bm25_qe_rm3= bm25_base >> pt.rewrite.RM3(index) >> bm25_base
#res_qe = bm25_qe_rm3.transform(topics)
#pt.Utils.evaluate(res_qe, qrels, metrics = ['ndcg',nDCG@10,'map'])


In [23]:
bm25_qe_bo1= pt.BatchRetrieve(index,wmodel="BM25",controls={"qe":"on"})
#res_qe = bm25_qe_bo1.transform(topics)
#pt.Utils.evaluate(res_qe, qrels, metrics = ['ndcg',nDCG@10,'map'])


In [24]:
#bm25 with doc2query
bm25_d2q=pt.BatchRetrieve(index_d2q,wmodel="BM25")
#res_d2q = bm25_d2q.transform(topics)
#pt.Utils.evaluate(res_d2q, qrels, metrics = ['ndcg',nDCG@10,'map'])

In [25]:
#bm25 with query expansion + doc2query
bm25_qe_rm3_d2q=bm25_d2q >> pt.rewrite.RM3(index_d2q) >> bm25_d2q
#res_qe_d2q = bm25_qe_rm3_d2q.transform(topics)
#pt.Utils.evaluate(res_qe_d2q, qrels, metrics = ['ndcg',nDCG@10,'map'])

In [26]:
bm25_qe_bo1_d2q=pt.BatchRetrieve(index_d2q,wmodel="BM25",controls={"qe":"on"})
#res_qe_d2q = bm25_qe_bo1_d2q.transform(topics)
#pt.Utils.evaluate(res_qe_d2q, qrels, metrics = ['ndcg',nDCG@10,'map'])

In [ ]:
#compare results
pt.Experiment([bm25_base,bm25_qe_bo1,bm25_qe_rm3,bm25_d2q,bm25_qe_bo1_d2q,bm25_qe_rm3_d2q]
             ,topics
             ,qrels
             ,['ndcg',nDCG@10,'map','mrt']
             ,names=['BM25 baseline','BM25-qe(Bo1)','BM25-qe(RM3)','BM25-d2q','BM25-qe(Bo1)+d2q','BM25-qe(RM3)+d2q'])

00:01:56.041 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
